# Socket

![socket](./img/img_socket.png)

두 프로그램이 네트워크상에서 데이터를 서로 송수신 할 수 있도록 양쪽에 생성되는 연결부이다.

소켓은 수행하는 역할에따라 두 종류의 소켓으로 나뉘는데

데이터를 요청하는 클라이언트 소켓, 그리고 클라이언트의 요청을받아 데이터를 서빙하는 서버 소켓으로 나뉜다.


## 소켓 API의 실행흐름

앞서 언급한 서버 소켓과 클라이언트 소켓은 다음과 같은 일련의과정을 통해서 연결을 수립하고 데이터 송수신을 수행한다.

![socket-flow](./img/img_socket_flow.png)

### 클라이언트 소켓

1. socket
    * 소켓을 생성하는 단계이다. 최초 소켓이 만들어지는 시점에는 어떠한 연결 대상 정보도 들어있지 않다.
2. connect
    * "IP주소"와 "포트번호"로 식별되는 서버 소켓에게 연결 요청을 보낸다.
    * connect()는 기본적으로 블로킹 방식으로 동작한다. 
    * 즉, 서버 소켓과 연결 요청에 대한 결과가 결정되기 전까지는 connect()의 실행이 끝나지 않는다는 것을 의미한다.
3. send / recv
    * 데이터를 송수신하는 단계다.
    * send(), recv()는 connect()와 마찬가지로 블로킹 방식으로 동작한다.
4. close
    * 데이터 송수신이 모두 완료되면 close()를 사용하여 소켓을 닫는다.
        
<br/>

### 서버 소켓

1. socket
    * 클라이언트 소켓과 동일하게 소켓을 생성하는 단계다.
2. bind
    * 운영체제가 특정 포트번호를 서버 소켓이 사용하도록 소켓과 포트 번호를 바인딩하는 단계다.
3. listen
    * 서버 소켓은 바인딩된 포트 번호로 클라이언트의 요청이 있는지 확인하며 대기상태에 머무른다.
    * 대기상태에서 빠져나오는 경우는 클라이언트로부터 요청이 수신되는 경우와, 에러가 발생되는 경우다.
      (연결 요청이 수신되었으면 SUCCESS, 에러가 발생하면 FAIL)
    * 클라이언트 연결요청이 들어오면 연결요청에 대한 정보는 시스템 내부적으로 관리되는 큐에 쌓이게된다.
4. accept
    * 최종적으로 클라이언트의 연결 요청을 받아들이는 단계다.
    * 큐에 쌓여있는 첫번째 연결요청을 매핑하여 이후의 단계를 수행한다.
5. send / recv
    * 클라이언트 소켓과 마찬가지로 데이터 송수신을 수행한다.
6. close
    * 클라이언트 소켓과 마찬가지로 close()를 통해 소켓을 닫는다.

<br/>

## 파이썬에서 Socket의 사용

우선 파이썬에서 제공하는 socket 모듈을 임포트한다.

In [3]:
import socket

### 클라이언트 소켓

In [ ]:
import socket

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(("127.0.0.1", 10000))

try:
    client_socket.send('Hello World!'.encode())
    msg = client_socket.recv(1024)
    print(msg.decode())

finally:
    client_socket.close()

#### 1. socket

socket 생성을위해 socket함수에 "주소 체계"와 "소켓 유형"에 대한 두가지 인자를 전달한다.

In [ ]:
client_socket = socket(socket.AF_INET, socket.SOCK_STREAM)

소켓이 만들어진 당시에는 인터넷 외에도 다른 프로토콜도 고려해서 만들어졌는데,<br/>
인터넷 기반의 통신을 사용하기 위해서는 `AF_INET`을 주소 체계의 인자로 전달하면 된다.

소켓 유형은 다음과 같이 3개로 분류 가능하다.
1. `SOCK_STREAM`
    * 연결형 서비스를 지원하며 AF_INET에서는 TCP 프로토콜을 사용
2. `SOCK_DGRM`
    * SOCK_DGRM은 비연결형 서비스를 지원하며 AF_INET에서는 UDP 프로토콜을 사용
3. `SOCK_RAW`
    * IP 프로토콜을 사용하는 방법이다.
    
소켓 유형에 대해서 요약하면 TCP를 사용하고 싶다면 SOCK_STREAM을 <br/>
UDP를 사용하고 싶다면 SOCK_DGRM을 쓰면된다.

#### 2. connect

서버 소켓의 IP와 Port번호가 포함된 튜플을 파라미터로 전달하여 서버 소켓과 연결을 시도한다.

connect를 호출하면 요청에 대한 결과가 결정되기전까지 블로킹된다.

In [ ]:
client_socket.connect(("127.0.0.1", 10000))

#### 3. send / recv

앞선단계에서 connect를 통해 연결 요청이 성공하면, 아래의 예시 코드처럼<br/>
send와 recv를 통해 서버 소켓과 데이터를 송수신 가능하다.

In [ ]:
try:
    client_socket.send('Hello World!'.encode())
    msg = client_socket.recv(1024)
    print(msg.decode())

#### 4. close

데이터 송수신이 완료되면 소켓을 닫는다.

In [ ]:
finally:
    clientSocket.close()

<br/>

### 서버 소켓

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('127.0.0.1', 10000))
server_socket.listen(5)
client_socket, addr = server_socket.accept()

while True:
    data = client_socket.recv(1024)
    if data:
        client_socket.sendall(data)
    else:
        break

client_socket.close()
server_socket.close()

#### 1. socket
클라이언트 소켓과 마찬가지로 서버소켓도 주소체계와 소켓유형을 설정하여 소켓을 생성한다.

In [ ]:
server_socket = socket(socket.AF_INET, socket.SOCK_STREAM)

#### 2. bind

방금 생성한 소켓에 특정포트를 바인딩한다.

In [ ]:
server_socket.connect(("127.0.0.1", 10000))

#### 3. listen

listen함수를 통해 클라이언트의 요청을 대기한다. 

함수 호출시 인자를 통해 처리가능한 최대 연결의 수를 지정할 수 있다.

In [ ]:
server_socket.listen(5)

#### 4. accept

클라이언트와 연결을 수립하며, 클라이언트 소켓과 클라이언트의 주소를 반환받는다.

In [ ]:
client_socket, addr = server_socket.accept()

#### 5. send / recv

클라이언트로부터 데이터를 수신받고 송신받는 동작을 수행하다가 <br/>
더이상 전송할 데이터가 없으면 데이터 송수신을 종료한다.

In [ ]:
while True:
    # 데이터 수신
    data = client_socket.recv(1024)
    
    # 데이터 송신 (여기서는 테스트를 위해서 수신받은 데이터 송신)
    if data:
        client_socket.sendall(data)
        
    # 전송할 데이터가 없으면 데이터송수신 종료
    else: 
        break

#### 6. close

클라이언트 소켓과 마찬가지로 서버 소켓도 데이터 송수신 완료후 소켓을 닫는다.

In [ ]:
client_socekt.close()
server_socket.close()